In [8]:
import pandas as pd

# Read the Excel file
df_excel = pd.read_excel("./preprocessed_data.xlsx")

# Save it as a CSV file
df_excel.to_csv("./data.csv", index=False)


In [1]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

In [4]:
df = pd.read_csv("./sample_data.csv")
df.head()

,item_id,order_status,created_at,stock_keeping_unit,price,qty_ordered,grand_total,increment_id,product_category,sales_commission_code,discount_amount,payment_method,BI Status,market_value,Customer Since,Customer ID
0,211133,canceled,2016-07-01,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,240.0,1,240.0,100147444,Beauty & Grooming,Unknown,0.0,Cash_on_delivery,Gross,240,2016-07-01,2
1,211134,canceled,2016-07-01,Ego_UP0017-999-MR0,2450.0,1,2450.0,100147445,Women's Fashion,Unknown,0.0,Cash_on_delivery,Gross,2450,2016-07-01,3
2,211135,completed,2016-07-01,kcc_krone deal,360.0,1,60.0,100147446,Beauty & Grooming,R-FSD-52352,300.0,Cash_on_delivery,Net,360,2016-07-01,4
3,211136,refund,2016-07-01,BK7010400AG,555.0,2,1110.0,100147447,Soghaat,Unknown,0.0,Cash_on_delivery,Valid,1110,2016-07-01,5
4,211137,canceled,2016-07-01,UK_Namkino All In One 200 Gms,80.0,1,80.0,100147448,Soghaat,Unknown,0.0,Cash_on_delivery,Gross,80,2016-07-01,6


In [5]:
# Initialize Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("E-Commerce Dashboard", style={"textAlign": "center"}),

    # Dropdown for selecting chart type
    html.Div([
        html.Label("Select Chart Type:"),
        dcc.Dropdown(
            id="chart-type",
            options=[
                {"label": "Order Status Distribution", "value": "status"},
                {"label": "Sales by Category", "value": "category"},
                {"label": "Payment Method Distribution", "value": "payment"},
            ],
            value="status",
        ),
    ], style={"width": "50%", "margin": "0 auto"}),

    # Graph output
    dcc.Graph(id="chart-output"),
])

# Callback for updating chart
@app.callback(
    Output("chart-output", "figure"),
    Input("chart-type", "value")
)
def update_chart(chart_type):
    if chart_type == "status":
        fig = px.pie(
            df, names="order_status", title="Order Status Distribution"
        )
    elif chart_type == "category":
        fig = px.bar(
            df.groupby("product_category")[["grand_total"]].sum().reset_index(),
            x="product_category",
            y="grand_total",
            title="Sales by Category",
        )
    elif chart_type == "payment":
        fig = px.pie(
            df, names="payment_method", title="Payment Method Distribution"
        )
    else:
        fig = px.scatter()

    return fig

# Run server
if __name__ == "__main__":
    app.run_server(debug=True)
